In [1]:
import os
import sys
sys.path.append("../../")
from imp import reload

import Reader

In [2]:
reload(Reader)

<module 'Reader' from '../..\\Reader.py'>

In [3]:
feature_path = r"D:\Workspace\python_workspace\gearbox-fault-detection\local\feature\niu_jia_ling"
speed_path = r"D:\Workspace\python_workspace\gearbox-fault-detection\local\rotating_speed\niu_jia_ling"
wind_turbine = "2"
sensors = (
    "gearbox",
    "low_speed_shaft",
    "high_speed_shaft",
)

In [7]:
reader = Reader.Reader()
feature = reader.read_feature(os.path.join(feature_path, wind_turbine), sensors)
speed = reader.read_speed(os.path.join(speed_path, wind_turbine), sensors)

In [8]:
feature

gearbox                                               \
                         mean             sd           root            rms   
date                                                                         
2018-05-01 00:00  4679.577266   86221.264502   58192.544768   86346.479377   
2018-05-01 01:00  4806.902656  205091.237849  141714.930088  205143.557277   
2018-05-01 02:00  4639.075742   80573.135745   57310.279606   80705.003764   
2018-05-01 03:00  4550.468867   69759.623197   46245.310388   69906.521166   
2018-05-01 04:00  4631.151680  162831.883695  108891.212721  162894.549331   
...                       ...            ...            ...            ...   
2019-11-01 18:00  4137.562109   79686.322831   53291.913141   79792.113790   
2019-11-01 19:00  4260.303516   84799.782349   62668.079608   84905.078610   
2019-11-01 20:00  4224.751680  197151.464868  136680.622881  197192.875931   
2019-11-01 21:00  4335.974453  207530.048733  145436.897804  207571.287585   
2019-11-01 23:00  4426.285586  162796.974570  111009.402042  162853.958090   

                                                                              \
                      peak  skewness  kurtosis     crest clearance     shape   
date                                                                           
2018-05-01 00:00  466614.0 -0.006183  0.354838  5.403972  8.018450  1.257220   
2018-05-01 01:00  994263.0 -0.147876 -0.079250  4.846669  7.015937  1.237067   
2018-05-01 02:00  361023.0  0.137856 -0.373456  4.473366  6.299446  1.217310   
2018-05-01 03:00  409241.0 -0.016443  0.740456  5.854118  8.849351  1.273548   
2018-05-01 04:00  979004.0 -0.261065  0.516657  6.010048  8.990661  1.264507   
...                    ...       ...       ...       ...       ...       ...   
2019-11-01 18:00  455017.0 -0.007364  0.614254  5.702531  8.538200  1.266718   
2019-11-01 19:00  320435.0 -0.101863 -0.671417  3.774038  5.113209  1.188648   
2019-11-01 20:00  889282.0 -0.038443 -0.105922  4.509707  6.506277  1.235051   
2019-11-01 21:00  945740.0  0.046140 -0.257574  4.556218  6.502751  1.226501   
2019-11-01 23:00  790405.0 -0.144892  0.063142  4.853459  7.120163  1.247570   

                  ... high_speed_shaft                              \
                  ...               p5             p6           p7   
date              ...                                                
2018-05-01 00:00  ...      3888.989758   91490.866360  5432.420939   
2018-05-01 01:00  ...      3748.948327  110272.156566  5177.821098   
2018-05-01 02:00  ...      4293.519586   79463.721463  5839.923740   
2018-05-01 03:00  ...      3941.763766   87925.425519  5481.630564   
2018-05-01 04:00  ...      3645.859410  108422.413599  5094.134900   
...               ...              ...            ...          ...   
2019-11-01 18:00  ...      3836.503617   87706.411148  5361.335433   
2019-11-01 19:00  ...      4284.611451   73619.429347  5770.614686   
2019-11-01 20:00  ...      3773.104751  114225.107455  5105.743951   
2019-11-01 21:00  ...      3753.055524  108898.896038  5145.465543   
2019-11-01 23:00  ...      3744.273214  103953.871377  5145.675550   

                                                                        \
                           p8        p9        p10       p11       p12   
date                                                                     
2018-05-01 00:00  9710.203803  0.559455  23.525613  0.036850  0.004100   
2018-05-01 01:00  9475.347996  0.546452  29.414158  0.032747  0.002857   
2018-05-01 02:00  9777.944796  0.597255  18.507828  0.032508  0.004976   
2018-05-01 03:00  9664.067234  0.567218  22.306112  0.036296  0.004297   
2018-05-01 04:00  9472.825442  0.537763  29.738506  0.034602  0.003023   
...                       ...       ...        ...       ...       ...   
2019-11-01 18:00  9613.678802  0.557678  22.861027  0.038236  0.004445   
2019-11-01 19:00  9613.986060  0.600231  17.182288  0.033853  0.005641   
2019-11-01 20

In [6]:
speed

,gearbox,low_speed_shaft,high_speed_shaft
,speed,speed,speed
date,,,
2018-05-01 00:00,1404.8,1404.8,1404.8
2018-05-01 01:00,1701.3,1701.3,1701.3
2018-05-01 02:00,1051.8,1051.8,1051.8
2018-05-01 03:00,1200.1,1200.1,1200.1
2018-05-01 04:00,1666.0,1666.0,1666.0
...,...,...,...
2019-11-01 18:00,1221.3,1221.3,1221.3
2019-11-01 19:00,1058.9,1058.9,1058.9
